In [1]:
# Mount Google Drive and set up paths + hyperparameters
from google.colab import drive
drive.mount('/content/drive')

import os
import tensorflow as tf

ROOT_DIR    = '/content/drive/Shareddrives/OmniClick Team'
DATA_DIR    = os.path.join(ROOT_DIR, 'datasets')
UNITY_DIR   = os.path.join(DATA_DIR, 'UnityEyes')   # must contain train/val/test

IMG_SIZE   = (96, 96)
BATCH_SIZE = 32
SEED       = 42028
AUTOTUNE   = tf.data.AUTOTUNE
EPOCHS     = 100

# Prepare weights folder
WEIGHTS_DIR  = os.path.join(ROOT_DIR, 'notebooks', 'weights')
os.makedirs(WEIGHTS_DIR, exist_ok=True)
WEIGHTS_PATH = os.path.join(WEIGHTS_DIR, 'unity_baseline_weights.h5')


Mounted at /content/drive


In [3]:
# 2.1) Load raw train & val datasets so we can read class_names
raw_train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(UNITY_DIR, 'train'),
    labels='inferred', label_mode='categorical',
    image_size=IMG_SIZE, batch_size=BATCH_SIZE,
    shuffle=True, seed=SEED
)
raw_val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(UNITY_DIR, 'val'),
    labels='inferred', label_mode='categorical',
    image_size=IMG_SIZE, batch_size=BATCH_SIZE,
    shuffle=False
)

# 2.2) Capture the class names and number of classes
class_names_unity = raw_train_ds.class_names
NUM_CLASSES_UNITY = len(class_names_unity)
print("UnityEyes classes:", class_names_unity)

# 2.3) Build the optimized pipelines
def preprocess(image, label):
    """Scale image pixels to [0,1]."""
    return tf.cast(image, tf.float32) / 255.0, label

train_ds = (
    raw_train_ds
    .map(preprocess, num_parallel_calls=AUTOTUNE)
    .cache().shuffle(1000, seed=SEED).prefetch(AUTOTUNE)
)
val_ds = (
    raw_val_ds
    .map(preprocess, num_parallel_calls=AUTOTUNE)
    .cache().prefetch(AUTOTUNE)
)


Found 61073 files belonging to 8 classes.
Found 72500 files belonging to 8 classes.
UnityEyes classes: ['BottomCenter', 'BottomLeft', 'BottomRight', 'MiddleLeft', 'MiddleRight', 'TopCenter', 'TopLeft', 'TopRight']


In [4]:
from tensorflow.keras import layers, models, Input

model = models.Sequential([
    Input(shape=(*IMG_SIZE, 3)),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128,3, activation='relu'),
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(NUM_CLASSES_UNITY, activation='softmax')
])

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 94, 94, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 47, 47, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 45, 45, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 22, 22, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 20, 20, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │         1,032 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 110,792 (432.78 KB)

 Trainable params: 110,792 (432.78 KB)

 Non-trainable params: 0 (0.00 B)

In [5]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)
]


In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks
)


Epoch 1/100


In [ ]:
model.save_weights(WEIGHTS_PATH)
print("Unity pre-trained weights saved to:", WEIGHTS_PATH)
